# WAR Data Explorer

Interactive exploration of flattened WAR (Windowed Analysis Results) data using ExperimentPlotter.

This notebook loads aggregated WAR data from multiple animals and provides interactive visualization for:
- Statistical comparisons across genotypes and experimental conditions
- Feature distribution analysis (linear, band, connectivity features)
- Cross-animal statistical analysis
- Frequency-domain and connectivity analysis

**Data Source**: Flattened WARs from `results/wars_flattened/{animal}/war.pkl`

## Setup and Data Loading

In [ ]:
import sys
import warnings
from pathlib import Path
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

# Add pythoneeg to path for Snakemake execution
sys.path.insert(0, str(Path("../../pythoneeg").resolve()))

from pythoneeg import visualization, constants
from pythoneeg.visualization.plotting import ExperimentPlotter

# Configure matplotlib and seaborn
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

In [ ]:
# Data loading: Handle both Snakemake and standalone execution
try:
    # When run via Snakemake, use snakemake object
    war_files = snakemake.input
    print(f"Running via Snakemake with {len(war_files)} WAR files")
except NameError:
    # When run standalone, discover files manually
    war_files = list(Path("../../results/wars_flattened").glob("*/war.pkl"))
    print(f"Running standalone with {len(war_files)} discovered WAR files")

if not war_files:
    raise FileNotFoundError("No WAR files found. Make sure flattened WARs have been generated.")
    
print(f"Found {len(war_files)} WAR files to load")

In [ ]:
# Load all WAR files
wars = []
failed_loads = []

for war_file in war_files:
    try:
        war_path = Path(war_file)
        animal_dir = war_path.parent
        
        # Load WAR using the standard method
        war = visualization.WindowAnalysisResult.load_pickle_and_json(
            folder_path=animal_dir,
            pickle_name="war.pkl",
            json_name="war.json"
        )
        wars.append(war)
        print(f"✓ Loaded {animal_dir.name}: {war.animal_id}, genotype: {war.genotype}")
        
    except Exception as e:
        failed_loads.append((war_file, str(e)))
        print(f"✗ Failed to load {war_file}: {e}")

print(f"\nSuccessfully loaded {len(wars)} WAR objects")
if failed_loads:
    print(f"Failed to load {len(failed_loads)} files")

## Dataset Overview

In [ ]:
if not wars:
    raise ValueError("No WAR objects loaded successfully")

# Create ExperimentPlotter for cross-animal analysis
ep = ExperimentPlotter(wars, use_abbreviations=True)

print("=== Dataset Overview ===")
print(f"Number of animals: {len(wars)}")
print(f"Animals: {[war.animal_id for war in wars]}")
print(f"Genotypes: {[war.genotype for war in wars]}")
print(f"Channel names: {ep.all_channel_names}")

# Show data structure of first WAR
sample_war = wars[0]
print(f"\n=== Sample WAR Structure (Animal: {sample_war.animal_id}) ===")
print(f"Result shape: {sample_war.result.shape}")
print(f"Columns: {list(sample_war.result.columns)}")
print(f"Available features: {sample_war._feature_columns}")
print(f"Non-feature columns: {sample_war._nonfeature_columns}")

In [ ]:
# Feature categories overview
print("=== Feature Categories ===")
print(f"Linear features: {constants.LINEAR_FEATURES}")
print(f"Band features: {constants.BAND_FEATURES}")
print(f"Matrix features: {constants.MATRIX_FEATURES}")
print(f"Histogram features: {constants.HIST_FEATURES}")

# Check which features are actually present in the data
available_features = sample_war._feature_columns
print(f"\nFeatures present in data: {available_features}")

# Categorize available features
present_linear = [f for f in constants.LINEAR_FEATURES if f in available_features]
present_band = [f for f in constants.BAND_FEATURES if f in available_features]
present_matrix = [f for f in constants.MATRIX_FEATURES if f in available_features]
present_hist = [f for f in constants.HIST_FEATURES if f in available_features]

print(f"Present linear features: {present_linear}")
print(f"Present band features: {present_band}")
print(f"Present matrix features: {present_matrix}")
print(f"Present histogram features: {present_hist}")

## Genotype and Experimental Condition Analysis

In [ ]:
# Genotype distribution
genotype_counts = pd.Series([war.genotype for war in wars]).value_counts()
display(Markdown("### Genotype Distribution"))
display(genotype_counts)

# Create summary dataframe for analysis
summary_data = []
for war in wars:
    # Extract basic info
    info = {
        'animal_id': war.animal_id,
        'genotype': war.genotype,
        'n_timepoints': len(war.result),
        'animaldays': len(war.animaldays)
    }
    
    # Add sex and gene categories (following EP figures script pattern)
    if war.genotype:
        info['sex'] = "Male" if war.genotype.startswith('M') else "Female" if war.genotype.startswith('F') else "Unknown"
        if war.genotype in ["MWT", "FWT"]:
            info['gene'] = "WT"
        elif war.genotype in ["MHet", "FHet"]:
            info['gene'] = "Het"
        elif war.genotype in ["MMut", "FMut"]:
            info['gene'] = "Mut"
        else:
            info['gene'] = war.genotype
    else:
        info['sex'] = "Unknown"
        info['gene'] = "Unknown"
    
    summary_data.append(info)

summary_df = pd.DataFrame(summary_data)
display(Markdown("### Animal Summary"))
display(summary_df)

# Group by experimental conditions
display(Markdown("### Experimental Groups"))
group_summary = summary_df.groupby(['sex', 'gene']).agg({
    'animal_id': 'count',
    'n_timepoints': ['mean', 'std'],
    'animaldays': ['mean', 'std']
}).round(2)
group_summary.columns = ['N_Animals', 'Mean_Timepoints', 'Std_Timepoints', 'Mean_Days', 'Std_Days']
display(group_summary)

## Linear Feature Analysis

Analysis of basic linear features (RMS, amplitude variance, PSD total, etc.)

In [ ]:
# Analyze linear features if available
if present_linear:
    # Plot RMS amplitude by genotype
    if 'rms' in present_linear:
        fig, ax, stats = ep.plot_boxplot(
            feature='rms',
            xgroup='genotype',
            channels='all',
            remove_outliers='iqr',
            show_outliers=True,
            title='RMS Amplitude by Genotype'
        )
        plt.show()
    
    # Plot amplitude variance by genotype
    if 'ampvar' in present_linear:
        fig, ax, stats = ep.plot_boxplot(
            feature='ampvar',
            xgroup='genotype',
            channels='all',
            title='Amplitude Variance by Genotype'
        )
        plt.show()
    
    # Plot PSD total power
    if 'psdtotal' in present_linear:
        fig, ax, stats = ep.plot_boxplot(
            feature='psdtotal',
            xgroup='genotype',
            channels='all',
            title='Total PSD Power by Genotype'
        )
        plt.show()

else:
    print("No linear features available in the dataset")

## Band-Specific Feature Analysis

Analysis of frequency band features (delta, theta, alpha, beta, gamma)

In [ ]:
if present_band:
    # Plot band power features
    if 'psdband' in present_band:
        fig, ax, stats = ep.plot_boxplot(
            feature='psdband',
            xgroup='genotype',
            title='PSD Band Power by Genotype and Frequency Band'
        )
        plt.show()
    
    # Plot fractional band power
    if 'psdfrac' in present_band:
        fig, ax, stats = ep.plot_boxplot(
            feature='psdfrac',
            xgroup='genotype',
            title='Fractional PSD Power by Genotype and Frequency Band'
        )
        plt.show()
    
    # Log-transformed features
    if 'logpsdband' in present_band:
        fig, ax, stats = ep.plot_violin(
            feature='logpsdband',
            xgroup='genotype',
            title='Log PSD Band Power Distribution by Genotype'
        )
        plt.show()
        
else:
    print("No band features available in the dataset")

## Connectivity Analysis

Analysis of connectivity features (coherence, correlation)

In [ ]:
if present_matrix:
    # Plot correlation matrices
    if 'pcorr' in present_matrix:
        fig, ax, stats = ep.plot_2d_feature(
            feature='pcorr',
            xgroup='genotype',
            title='Pearson Correlation by Genotype'
        )
        plt.show()
    
    # Plot Fisher Z-transformed correlations
    if 'zpcorr' in present_matrix:
        fig, ax, stats = ep.plot_2d_feature(
            feature='zpcorr',
            xgroup='genotype',
            title='Fisher Z-transformed Correlation by Genotype'
        )
        plt.show()
    
    # Plot coherence if available
    if 'cohere' in present_matrix:
        # Overall coherence
        fig, ax, stats = ep.plot_2d_feature(
            feature='cohere',
            xgroup='genotype',
            title='Coherence by Genotype'
        )
        plt.show()
        
        # Frequency-specific coherence
        fig, ax, stats = ep.plot_2d_feature_freq(
            feature='cohere',
            xgroup='genotype',
            title='Coherence by Genotype and Frequency Band'
        )
        plt.show()
    
    # Plot imaginary coherence if available
    if 'imcoh' in present_matrix:
        fig, ax, stats = ep.plot_2d_feature_freq(
            feature='imcoh',
            xgroup='genotype',
            title='Imaginary Coherence by Genotype and Frequency Band'
        )
        plt.show()
        
else:
    print("No matrix/connectivity features available in the dataset")

## Cross-Feature Analysis

In [ ]:
# Feature correlation analysis
if len(present_linear) >= 2:
    # Get feature data for correlation analysis
    try:
        # Extract data for all animals
        all_data = []
        for war in wars:
            war_data = war.get_result(present_linear[:4], allow_missing=True)  # Limit to first 4 features
            war_data['animal_id'] = war.animal_id
            war_data['genotype'] = war.genotype
            all_data.append(war_data)
        
        combined_data = pd.concat(all_data, ignore_index=True)
        
        # Compute correlation matrix for linear features
        feature_corr = combined_data[present_linear[:4]].corr()
        
        plt.figure(figsize=(8, 6))
        sns.heatmap(feature_corr, annot=True, cmap='coolwarm', center=0, 
                    square=True, fmt='.2f')
        plt.title('Feature Correlation Matrix')
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Could not compute feature correlations: {e}")
        
else:
    print("Not enough linear features for correlation analysis")

## Custom Analysis Section

This section can be customized for specific research questions.

In [ ]:
# Custom analysis placeholder
# Users can add their own analysis here

print("=== Custom Analysis Placeholder ===")
print("Add your custom analysis code here.")
print("Available methods:")
print("- ep.plot_boxplot(feature, xgroup, ...)")
print("- ep.plot_violin(feature, xgroup, ...)")
print("- ep.plot_scatter(feature, xgroup, ...)")
print("- ep.plot_2d_feature(feature, xgroup, ...)")
print("- ep.plot_2d_feature_freq(feature, xgroup, ...)")
print("\nAvailable features:", available_features)
print("Available grouping options: 'animal', 'genotype', 'animal_id'")

## Summary and Export

In [ ]:
# Summary statistics
print("=== Analysis Summary ===")
print(f"Total animals analyzed: {len(wars)}")
print(f"Total features analyzed: {len(available_features)}")
print(f"Genotypes represented: {list(genotype_counts.index)}")

# Optional: Save summary data
try:
    # When run via Snakemake, save to logs directory
    output_dir = Path("../../logs/notebooks")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    summary_df.to_csv(output_dir / "war_explorer_summary.csv", index=False)
    print(f"Summary data saved to {output_dir / 'war_explorer_summary.csv'}")
    
except Exception as e:
    print(f"Could not save summary data: {e}")

print("\n=== WAR Data Explorer Analysis Complete ===")